##Домашнее задание к лекции "Основы веб-скрапинга"

##Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def get_habr_posts(query):
    habr_posts = pd.DataFrame()
    for q in query:
        URL = 'https://habr.com/ru/search/?'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)

        if req.status_code != 200:
            print(f"Failed to fetch results for query: {q}")
            continue

        soup = BeautifulSoup(req.text, 'html.parser')

        news = soup.find_all('article', class_='tm-articles-list__item')

        for el in news:
            title = el.find('a','tm-title__link').text
            link = el.find('a','tm-title__link').get('href')
            date = el.find('a', 'tm-article-datetime-published tm-article-datetime-published_link').text
            
            if 'megapost' in el.get('class', []):
                title += ' (megapost)' 

            if 'remote-blog' in el.get('class', []):
                print(f"Skipping remote blog post: {title}")
                continue

            
            row = {'Дата': date, 'Заголовок': title, 'Ссылка': 'https://habr.com' + link}
            habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])

    return habr_posts.reset_index(drop=True)

res = get_habr_posts(['python'])
res

,Дата,Заголовок,Ссылка
0,31 янв 2023 в 16:48,Скучный Python: повышаем качество кода,https://habr.com/ru/companies/otus/articles/71...
1,25 июл 2023 в 13:43,Автоматизация обеспечения качества кода на язы...,https://habr.com/ru/companies/otus/articles/75...
2,28 ноя 2018 в 06:00,Транспайлер-цепь Python → 11l → C++ [для ускор...,https://habr.com/ru/articles/431318/
3,7 апр 2023 в 13:02,Декораторы Python: пошаговое руководство,https://habr.com/ru/companies/otus/articles/72...
4,18 янв в 11:21,Уникальные стили визуализации данных в Python:...,https://habr.com/ru/articles/787210/
5,21 апр 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
6,27 июл 2023 в 14:21,Введение в Locust: open source инструмент для ...,https://habr.com/ru/companies/otus/articles/75...
7,4 дек 2020 в 21:03,Python как компилируемый статически типизирова...,https://habr.com/ru/news/531402/
8,3 мар 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...
9,3 июн 2020 в 09:00,Вышла версия 0.0.2 snakeware — дистрибутива Li...,https://habr.com/ru/news/505096/
